In [23]:
import pandas as pd
df=pd.read_csv(r'C:\Users\KARAN\Desktop\BBS Sir Proj\glass-clustering.csv',encoding='unicode_escape')
df

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0
...,...,...,...,...,...,...,...,...,...
208,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0
209,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0
210,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0
211,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0


In [24]:
from sklearn.model_selection import train_test_split
## we will do the splitting using a random state to ensure same splitting every time
X_train, X_test = train_test_split(df.values,test_size = .3,random_state = 13)

In [25]:
X_train

array([[ 1.51977, 13.81   ,  3.58   , ...,  8.67   ,  0.69   ,  0.     ],
       [ 1.52196, 14.36   ,  3.85   , ...,  9.15   ,  0.     ,  0.     ],
       [ 1.5159 , 13.24   ,  3.34   , ...,  8.22   ,  0.     ,  0.     ],
       ...,
       [ 1.5159 , 13.02   ,  3.58   , ...,  7.96   ,  0.     ,  0.     ],
       [ 1.51937, 13.79   ,  2.41   , ...,  9.77   ,  0.     ,  0.     ],
       [ 1.51594, 13.09   ,  3.52   , ...,  8.05   ,  0.     ,  0.09   ]])

In [26]:
X_test

array([[1.51590e+00, 1.28200e+01, 3.52000e+00, 1.90000e+00, 7.28600e+01,
        6.90000e-01, 7.97000e+00, 0.00000e+00, 0.00000e+00],
       [1.51640e+00, 1.43700e+01, 0.00000e+00, 2.74000e+00, 7.28500e+01,
        0.00000e+00, 9.45000e+00, 5.40000e-01, 0.00000e+00],
       [1.51602e+00, 1.48500e+01, 0.00000e+00, 2.38000e+00, 7.32800e+01,
        0.00000e+00, 8.76000e+00, 6.40000e-01, 9.00000e-02],
       [1.51888e+00, 1.49900e+01, 7.80000e-01, 1.74000e+00, 7.25000e+01,
        0.00000e+00, 9.95000e+00, 0.00000e+00, 0.00000e+00],
       [1.51813e+00, 1.34300e+01, 3.98000e+00, 1.18000e+00, 7.24900e+01,
        5.80000e-01, 8.15000e+00, 0.00000e+00, 0.00000e+00],
       [1.51652e+00, 1.35600e+01, 3.57000e+00, 1.47000e+00, 7.24500e+01,
        6.40000e-01, 7.96000e+00, 0.00000e+00, 0.00000e+00],
       [1.51831e+00, 1.43900e+01, 0.00000e+00, 1.82000e+00, 7.28600e+01,
        1.41000e+00, 6.47000e+00, 2.88000e+00, 0.00000e+00],
       [1.51730e+00, 1.23500e+01, 2.72000e+00, 1.63000e+00, 7.

In [27]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tensor_train = torch.FloatTensor(X_train).to(device)
tensor_test=torch.FloatTensor(X_test).to(device)

In [28]:
from autoencoder import Autoencoder
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import TensorDataset,DataLoader
inputs_dim=df.shape[1]
n_bottleneck=10
model=Autoencoder(inputs_dim,n_bottleneck).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
train_loader = DataLoader(TensorDataset(tensor_train, tensor_train), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(tensor_test, tensor_test), batch_size=64, shuffle=False)

In [29]:
from tqdm import tqdm
import os

lowest_val_loss = float('inf')  # Initialize with a large value
best_epoch = 0

num_epochs = 1000
for epoch in range(num_epochs):
    # Training
    model.train()
    total_train_loss = 0.0
    with tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs} - Training', unit='batch') as t:
        for batch in t:
            input_data, _ = batch
            input_data = input_data.to(device)

            optimizer.zero_grad()
            output_data = model(input_data)
            loss = criterion(output_data, input_data)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()
            t.set_postfix(loss=f'{total_train_loss / len(train_loader):.5f}')

    # Validation
    model.eval()
    total_val_loss = 0.0
    with torch.no_grad():
        with tqdm(val_loader, desc=f'Epoch {epoch + 1}/{num_epochs} - Validation', unit='batch') as t:
            for batch in t:
                val_input_data, _ = batch
                val_input_data = val_input_data.to(device)

                val_output_data = model(val_input_data)
                val_loss = criterion(val_output_data, val_input_data)
                total_val_loss += val_loss.item()
                t.set_postfix(loss=f'{total_val_loss / len(val_loader):.5f}')

    # Save model if validation loss is the lowest
    if total_val_loss < lowest_val_loss:
        lowest_val_loss = total_val_loss
        best_epoch = epoch + 1  # Epochs are 0-indexed, so add 1

        # Save the model's state dictionary and other relevant information
        checkpoint_path = 'best_model_checkpoint_glass.pth'
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'lowest_val_loss': lowest_val_loss,
        }, checkpoint_path)

print(f"Best model found at epoch {best_epoch} with validation loss: {lowest_val_loss:.5f}")


Epoch 1/1000 - Training:   0%|          | 0/3 [00:00<?, ?batch/s]

Epoch 1000/1000 - Validation: 100%|██████████| 1/1 [00:00<00:00, 163.82batch/s, loss=0.45316]

Best model found at epoch 998 with validation loss: 0.44597


In [30]:
checkpoint_path = 'best_model_checkpoint_glass.pth'
checkpoint = torch.load(checkpoint_path)
model = Autoencoder(inputs_dim, n_bottleneck).to(device)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [31]:
tensor_data=torch.FloatTensor(df.values).to(device)
df_encoded=model.encoder(tensor_data)
df_encoded_cpu = df_encoded.cpu().detach().numpy()

In [32]:
import umap.umap_ as umap
df_umapped = umap.UMAP(n_components = n_bottleneck / 2, 
                           metric = "euclidean",
                           n_neighbors = 50, 
                           min_dist = 0.0,
                           random_state = 13).fit_transform(df_encoded_cpu)

c:\Users\KARAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [33]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

# Assuming you have your review_umapped data
kmeans = KMeans(n_clusters=6, random_state=42)
cluster_labels = kmeans.fit_predict(df_umapped)

# Silhouette Score
silhouette_avg = silhouette_score(df_umapped, cluster_labels)
print(f'Silhouette Score: {silhouette_avg}')

# Davies-Bouldin Index
db_index = davies_bouldin_score(df_umapped, cluster_labels)
print(f'Davies-Bouldin Index: {db_index}')

# Calinski-Harabasz Index
ch_index = calinski_harabasz_score(df_umapped, cluster_labels)
print(f'Calinski-Harabasz Index: {ch_index}')


Silhouette Score: 0.573562741279602
Davies-Bouldin Index: 0.645876494800664
Calinski-Harabasz Index: 1316.5607063137945


c:\Users\KARAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
